# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [4]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

#!import plugins/StaticTextPlugin.cs
kernel.ImportPluginFromType<StaticTextPlugin>();

Define your ASK. What do you want the Kernel to do?

In [46]:
#pragma warning disable SKEXP0060

var ask = "Gumawa ka ng dalawang tula tungkol sa GinPom cocktail, tapos isalin mo sa English.";
var plan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine(plan);

{{!-- Step 1: Create two poems about GinPom cocktail --}}
{{set "poem1" (WriterPlugin-ShortPoem input="Ang GinPom cocktail, isang inumin ng kasiyahan, nagdudulot sigla sa bawat saluhan.")}}
{{set "poem2" (WriterPlugin-ShortPoem input="Sa bawat lagok ng GinPom cocktail, sumasabay ang tibok, naglalakbay sa mundo ng pag-ibig at pagkakaisa.")}}

{{!-- Step 2: Translate the first poem to English --}}
{{set "translatedPoem1" (WriterPlugin-Translate input=poem1 language="English")}}

{{!-- Step 3: Translate the second poem to English --}}
{{set "translatedPoem2" (WriterPlugin-Translate input=poem2 language="English")}}

{{!-- Step 4: Output the translated poems --}}
{{json "First Poem in English:"}}
{{json translatedPoem1}}
{{json "Second Poem in English:"}}
{{json translatedPoem2}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [47]:
#pragma warning disable SKEXP0060

var result = await plan.InvokeAsync(kernel);
result


First Poem in English:
In a town, there's a unique drink,
The GinPom cocktail, truly distinctive.
A mix of gin and pomelo, fresh and delicious,
With every sip, your smile is guaranteed.
Second Poem in English:
In a glass of GinPom, the heartbeat dances,  
Love and unity, travel in every gulp.  
With every sip and swig, the world becomes colorful.